In [0]:
dbutils.widgets.text("entityName", "", "Entity Name")
dbutils.widgets.text("dataSourceName", "", "Data Source Name")
dbutils.widgets.text("version", "", "Version")
dbutils.widgets.text("inputPath", "", "Input path")
dbutils.widgets.text("inputContainer", "", "Input container")
dbutils.widgets.text("outputPath", "", "Output path")
dbutils.widgets.text("outputContainer", "", "Output container")
entityName = dbutils.widgets.get("entityName")
dataSourceName = dbutils.widgets.get("dataSourceName")
version = dbutils.widgets.get("version")
inputPath = dbutils.widgets.get("inputPath")
inputContainer = dbutils.widgets.get("inputContainer")
outputPath = dbutils.widgets.get("outputPath")
outputContainer = dbutils.widgets.get("outputContainer")

In [0]:
%run  "./common"

In [0]:
source = get_path(inputContainer, inputPath)
data = spark.read.parquet(source)
data.show()

In [0]:
d = ({"name" : entityName, 
     "namespace" : dataSourceName + '.' + entityName ,
     "count" : str(data.count()), 
     "dimensions" : [
       { "name" : "Engine Name", "value" : engine['engineName'] } ,
       { "name" : "Entity Name", "value" : entityName } ,
       { "name" : "Data Source Name", "value" : dataSourceName } ,
       { "name" : "Version", "value" : version }
     ] 
    })
print(d)

add_metric(json.dumps(d))

In [0]:
stats = {x[0]: {'type': x[1]} for x in data.dtypes}

for attribute in data.columns:
  if stats[attribute]['type'] == 'int' or  'decimal' in stats[attribute]['type'] or 'float' in stats[attribute]['type'] or 'double' in  stats[attribute]['type'] or 'long' in  stats[attribute]['type']:
    sumof = data.select(sum(col(attribute)).cast(DoubleType()).alias('__sum__'), 
                        count(col(attribute)).cast(IntegerType()).alias('__count__'), 
                        min(col(attribute)).cast(IntegerType()).alias('__min__'), 
                        max(col(attribute)).cast(IntegerType()).alias('__max__'), 
                       
                       ).collect()
    sumof = sumof[0]
    
    d = ({"name" : attribute, 
     "sum" : str(sumof['__sum__']), 
     "count" : str(sumof['__count__']), 
     "min" : str(sumof['__min__']), 
     "max" : str(sumof['__max__']), 
     "namespace" : dataSourceName + '.' + entityName, 
     "dimensions" : [
       { "name" : "Engine Name", "value" : engine['engineName'] } ,
       { "name" : "Entity Name", "value" : entityName } ,
       { "name" : "Data Source Name", "value" : dataSourceName } ,
       { "name" : "Version", "value" : version } ,
       { "name" : "Column Name", "value" : attribute } ,
     ] 
    })
    print(d)
    add_metric(json.dumps(d))


In [0]:
output = get_path(outputContainer, outputPath)
data.write.format("delta").mode("overwrite").save(output)